In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import TensorBoard
import time
import tensorflow.keras.backend as K
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Lambda


In [10]:
# Implementing MobileFaceNet
def DepthwiseConv2D(x, filters, kernel_size, strides, padding='same', use_bias=False, name=None):
    x = tf.keras.layers.DepthwiseConv2D(kernel_size, strides=strides, padding=padding, use_bias=use_bias, name=name)(x)
    x = tf.keras.layers.BatchNormalization(name=name + '_bn')(x)
    x = tf.keras.layers.ReLU(6., name=name + '_relu')(x)
    x = tf.keras.layers.Conv2D(filters, kernel_size=1, padding='same', use_bias=False, name=name + '_conv')(x)
    x = tf.keras.layers.BatchNormalization(name=name + '_conv_bn')(x)
    x = tf.keras.layers.ReLU(6., name=name + '_conv_relu')(x)
    return x;

def MobileFaceNet(input_shape=(112, 112, 3), embedding_size=128):
    input = Input(shape=input_shape)
    x = Conv2D(64, kernel_size=3, strides=2, padding='same', use_bias=False, name='conv1')(input)
    x = tf.keras.layers.BatchNormalization(name='conv1_bn')(x)
    x = tf.keras.layers.ReLU(6., name='conv1_relu')(x)
    x = DepthwiseConv2D(x, 64, 3, 1, name='conv_dw_2')
    x = DepthwiseConv2D(x, 64, 3, 1, name='conv_dw_3')
    x = DepthwiseConv2D(x, 128, 3, 2, name='conv_dw_4')
    x = DepthwiseConv2D(x, 128, 3, 1, name='conv_dw_5')
    x = DepthwiseConv2D(x, 128, 3, 1, name='conv_dw_6')
    x = DepthwiseConv2D(x, 128, 3, 1, name='conv_dw_7')
    x = DepthwiseConv2D(x, 128, 3, 1, name='conv_dw_8')
    x = DepthwiseConv2D(x, 128, 3, 1, name='conv_dw_9')
    x = DepthwiseConv2D(x, 256, 3, 2, name='conv_dw_10')
    x = DepthwiseConv2D(x, 256, 3, 1, name='conv_dw_11')
    x = DepthwiseConv2D(x, 256, 3, 1, name='conv_dw_12')
    x = DepthwiseConv2D(x, 512, 3, 2, name='conv_dw_13')
    x = DepthwiseConv2D(x, 512, 3, 1, name='conv_dw_14')
    x = DepthwiseConv2D(x, 512, 3, 1, name='conv_dw_15')
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Dense(embedding_size, use_bias=False, name='embeddings')(x)
    x = tf.keras.layers.Lambda(lambda  x: tf.keras.backend.l2_normalize(x, axis=1),
                                 name='embeddings_norm')(x) 
    model = Model(inputs=input, outputs=x)
    return model;

model=MobileFaceNet()
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 112, 112, 3)]     0         
                                                                 
 conv1 (Conv2D)              (None, 56, 56, 64)        1728      
                                                                 
 conv1_bn (BatchNormalizatio  (None, 56, 56, 64)       256       
 n)                                                              
                                                                 
 conv1_relu (ReLU)           (None, 56, 56, 64)        0         
                                                                 
 conv_dw_2 (DepthwiseConv2D)  (None, 56, 56, 64)       576       
                                                                 
 conv_dw_2_bn (BatchNormaliz  (None, 56, 56, 64)       256       
 ation)                                                    

In [11]:
# Training the ModebileFaceNet on LFW Dataset
